类元编程是指在运行时创建或定制类的技艺。

在 Python 中，类是一等对象，因此任何时候都可以使用函数新建类，而无需使用 class 关键字。类装饰器也是函数，不过能够审查、修改、甚至把被装饰的类替换成其它类。元类是类元编程最高级的工具：使用元类可以创建具有某种特质的全新类种，例如抽象基类。

In [1]:
def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split()  # 鸭子类型
    except AttributeError:  # 不能调用 .replace 或 .split 方法
        pass  # 假定 field_names 本就是标识符组成的序列
    field_names = tuple(field_names)  # 使用属性名构建元组。作为新建类的 __slots__ 属性

    def __init__(self, *args, **kwargs):  # 新建类的 __init__ 方法
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self):  # 把类的实例变成可迭代对象
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):  # 迭代 __slots__ 和 self，生成字符串表示形式
        values = ', '.join('{}={!r}'.format(*i) for i
                           in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, values)

    cls_attrs = dict(__slots__ = field_names,  # 组件类属性字典
                     __init__  = __init__,
                     __iter__  = __iter__,
                     __repr__  = __repr__)

    return type(cls_name, (object,), cls_attrs)  # 调用 type 构造方法，构建新类

In [2]:
Dog = record_factory('Dog', 'name weight owner')  # 先写类名，后面跟着写在一个字符串里的多个属性名（空格或逗号分开）
rex = Dog('Rex', 30, 'Bob')
rex  # 字符串表示形式

Dog(name='Rex', weight=30, owner='Bob')

In [3]:
name, weight, _ = rex  # 可迭代对象
name, weight

('Rex', 30)

In [4]:
"{2}'s dog weighs {1}kg".format(*rex)  # 拆包

"Bob's dog weighs 30kg"

In [5]:
rex.weight = 32  # 可变对象
rex

Dog(name='Rex', weight=32, owner='Bob')

In [6]:
Dog.__mro__  # 继承自 object

(__main__.Dog, object)

type 的三个参数分别是 name、bases、dict ， dict 指定新类的属性名和值

```python
MyClass = type('MyClass', (MySuperClass, MyMixin),
              {'x':42, 'x2':lambda self: self.x * 2})
```

相当于

```python
class MyClass(MySuperClass, MyMixin):
    x = 42
    
    def x2(self):
        return self.x * 2
```

record_factory 函数创建的类，它的实例有个局限 —— 不能序列化，即不能使用 pickle 模块里的 dump/load 函数处理。

类装饰器与函数装饰器非常类似，是参数为类对象的函数，返回原来的类或修改后的类。

类装饰器有个重大缺点：只对直接依附的类有效。

In [7]:
import model_v6 as model

@model.entity  # 类装饰器
class LineItem:
    description = model.NonBlank()
    weight = model.Quantity()
    price = model.Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [8]:
# model_v6.py（部分）
def entity(cls):  # 装饰器的参数是一个类
    for key, attr in cls.__dict__.items():  # 迭代存储类属性的字典
        if isinstance(attr, Validated):  # 如果属性是 Validated 描述符的实例
            type_name = type(attr).__name__
            attr.storage_name = '_{}#{}'.format(type_name, key)  # 使用描述符类的名称和托管属性的名称
    return cls  # 返回修改后的类

In [9]:
raisins = LineItem('Golden raisins', 10, 6.95)
dir(raisins)[:3]

['_NonBlank#description', '_Quantity#price', '_Quantity#weight']

In [10]:
LineItem.description.storage_name

'_NonBlank#description'

In [11]:
raisins.description

'Golden raisins'

In [12]:
getattr(raisins, '_NonBlank#description')

'Golden raisins'

在导入时，计时器会从上到下一次性解析完 .py 模块的源码，然后生成用于执行的字节码。
- 如果句法有错误，就在此时报告
- 如果本地的 \_\_pycache__ 文件夹中有最新的 .pyc 文件，解释器就会跳过上述步骤

import 语句不只是声明，在进程中首次导入模块时，还会运行所导入模块中的全部顶层代码 —— 以后导入相同的模块则使用缓存，只做名称绑定。

解释器在导入时定义顶层函数，但是仅当在运行时调用函数才会执行函数的定义体。

在导入时，解释器会执行每个类的定义体，甚至会执行嵌套类的定义体。
- 执行类定义体的结果是，定义了类的属性和方法，并构建了类对象
- 从这个意义上理解，类的定义体属于“顶层代码”，因为它在导入时运行

1. 导入模块

先计算被装饰的类 ClassThree 的定义体，然后运行装饰器函数

- 解释器会执行所导入模块及其依赖（evalsupport）中的每个类定义体；
- 解释器先计算类的定义体，然后调用依附在类上的装饰器函数
- 只运行了一个用户定义的函数或方法：deco_alpha 装饰器

In [13]:
import evaltime

<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end
<[1]> evaltime module start
<[2]> ClassOne body
<[6]> ClassTwo body
<[7]> ClassThree body
<[200]> deco_alpha
<[9]> ClassFour body
<[14]> evaltime module end


2. 运行模块

类装饰器可能对子类没有影响

- deco_alpha 装饰器修改了 ClassThree.method_y 方法，因此要调用 three.method_y() 时会运行 inner_1 函数的定义体
- 只有程序结束时，绑定在全局变量 one 上的 ClassOne 实例才会被垃圾回收程序回收。

In [14]:
!python evaltime.py

<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end
<[1]> evaltime module start
<[2]> ClassOne body
<[6]> ClassTwo body
<[7]> ClassThree body
<[200]> deco_alpha
<[9]> ClassFour body
<[11]> ClassOne tests ..............................
<[3]> ClassOne.__init__
<[5]> ClassOne.method_x
<[12]> ClassThree tests ..............................
<[300]> deco_alpha:inner_1
<[13]> ClassFour tests ..............................
<[10]> ClassFour.method_y
<[14]> evaltime module end
<[4]> ClassOne.__del__


默认情况下，Python 中的类是 type 类的实例，也就是说 type 是大多数内置的类和用户定义的类的元类。为了避免无限回溯，type 是其自身的实例。

![](21-2.png)

In [15]:
'spam'.__class__

str

In [16]:
str.__class__

type

In [17]:
LineItem.__class__

type

In [18]:
type.__class__

type

object 类和 type 类之间的关系很独特：object 是 type 的实例，而 type 是 object 的子类。

除了 type ，标准库中还有一些别的元类，例如 ABCMeta 和 Enum ，collections.Iterable 所属的类是 abc.ABCMeta 。

所有类都是直接或间接地是 type 的实例，不过只有元类同时也是 type 的子类，元类从 type 类继承了构建类的能力，因此可以作为制造类的工厂。

具体来说，元类可以通过实现 \_\_init__ 方法定制实例，编写元类时，通常会把 self 参数改成 cls 。

![](21-3.png)

In [19]:
import collections

collections.Iterable.__class__

<ipython-input-19-d4fdfb4ce7d2>:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  collections.Iterable.__class__


abc.ABCMeta

In [20]:
import abc

abc.ABCMeta.__class__

type

In [21]:
abc.ABCMeta.__mro__

(abc.ABCMeta, type, object)

3. 导入模块

- 与 1 的关键区别是，创建 ClassFive 时调用了 MetaAleph.\_\_init__ 方法
- 创建 ClassFive 的子类 ClassSix 时也调用了 MetaAleph.\_\_init__ 方法

In [22]:
# 1. 中已经导入过 evalsupport.py 了，因此这里使用缓存，没有再次导入
import evaltime_meta.py

<[1]> evaltime_meta module start
<[2]> ClassThree body
<[200]> deco_alpha
<[4]> ClassFour body
<[6]> ClassFive body
<[500]> MetaAleph.__init__
<[9]> ClassSix body
<[500]> MetaAleph.__init__
<[15]> evaltime_meta module end


ModuleNotFoundError: No module named 'evaltime_meta.py'; 'evaltime_meta' is not a package

In [23]:
# evalsupport.py（部分）
class MetaAleph(type):
    print('<[400]> MetaAleph body')
    
    # 四个参数：初始化的对象，与构建类时传给 type 的参数一样
    def __init__(cls, name, bases, dic):
        print('<[500]> MetaAleph.__init__')

        def inner_2(self):
            print('<[600]> MetaAleph.__init__:inner_2')

        cls.method_z = inner_2

<[400]> MetaAleph body


4. 运行模块

- 装饰器依附到 ClassThree 类上之后，method_y 方法被替换成 inner_1 方法
- 没有依附装饰器的 ClassFour 类不受影响
- MetaAleph 类的 \_\_init__ 方法把 ClassFive.method_z 方法替换成 inner_2 函数
- ClassFive 的子类 ClassSix 也一样，method_z 方法替换成 inner_2 函数

In [24]:
!python evaltime_meta.py

<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end
<[1]> evaltime_meta module start
<[2]> ClassThree body
<[200]> deco_alpha
<[4]> ClassFour body
<[6]> ClassFive body
<[500]> MetaAleph.__init__
<[9]> ClassSix body
<[500]> MetaAleph.__init__
<[11]> ClassThree tests ..............................
<[300]> deco_alpha:inner_1
<[12]> ClassFour tests ..............................
<[5]> ClassFour.method_y
<[13]> ClassFive tests ..............................
<[7]> ClassFive.__init__
<[600]> MetaAleph.__init__:inner_2
<[14]> ClassSix tests ..............................
<[7]> ClassFive.__init__
<[600]> MetaAleph.__init__:inner_2
<[15]> evaltime_meta module end


定制描述符的元类

In [25]:
import model_v7 as model

class LineItem(model.Entity):  # model.Entity 的子类
    description = model.NonBlank()
    weight = model.Quantity()
    price = model.Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [26]:
raisins = LineItem('Golden raisins', 10, 6.95)
dir(raisins)[:3]

['_NonBlank#description', '_Quantity#price', '_Quantity#weight']

In [27]:
LineItem.description.storage_name

'_NonBlank#description'

In [28]:
raisins.description

'Golden raisins'

In [29]:
getattr(raisins, '_NonBlank#description')

'Golden raisins'

元类的特殊方法 \_\_prepare__ 只在元类中有用，而且必须声明为类方法（使用 @classmethod 装饰器定义）

In [30]:
import model_v8 as model

class LineItem(model.Entity):
    description = model.NonBlank()
    weight = model.Quantity()
    price = model.Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price


In [31]:
for name in LineItem.field_names():
    print(name)

description
weight
price


在现实世界中，框架和库会使用元类协助程序员执行很多任务，例如：
- 验证属性
- 一次把装饰器依附到多个方法上
- 序列化对象或转换数据
- 对象关系映射
- 基于对象的持久存储
- 动态转换使用其他语言编写的类结构

Python 数据模型为每个类定义了很多属性
- \_\_mro__
- \_\_class__
- \_\_name__
- \_\_bases__
    - 由类的基类组成的元组
- \_\_qualname__
    - 类或函数的限定名称，即从模块的全局作用域到类的淀粉路径
- \_\_subclasses__()
    - 返回包含类的直接子类的列表
- mro()
    - 获取存储在类属性 \_\_mro__ 中的超类元组

类元编程是指动态创建或制定类。

类装饰器使函数，其参数是被装饰的类，用于审查和修改刚创建的类，甚至替换成其他类。

元类可以定制类的层次结构。类装饰器只能影响一个类，而且对后代可能没有影响。